In [1]:
import math
import numpy as np
import numpy.random as npr
from numpy import linalg
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import Dataset
import torch.optim as optim
import time
from math import *
torch.cuda.set_device(0)

In [1]:
torch.set_default_tensor_type('torch.DoubleTensor')

NameError: name 'torch' is not defined

In [3]:
class ResNet(nn.Module):
    def __init__(self,dim, m,o):
        super(ResNet, self).__init__()
        self.Ix = torch.zeros([dim,m]).cuda()
        self.Ix[0,0] = 1
        self.Ix[1,1] = 1
        self.fc1 = nn.Linear(dim, m)
        self.fc2 = nn.Linear(m, m)
        
        self.fc3 = nn.Linear(m, m)
        self.fc4 = nn.Linear(m, m)

        
        self.outlayer = nn.Linear(m, o)

    def forward(self, x):
        s = x@self.Ix
        y = self.fc1(x)
        y = F.relu(y**deg)
        y = self.fc2(y)
        y = F.relu(y**deg)
        y = y+s
        
        s=y
        y = self.fc3(y)
        y = F.relu(y**deg)
        y = self.fc4(y)
        y = F.relu(y**deg)
        y = y+s
        
        
        output = self.outlayer(y)
        return output
deg = 2

In [4]:
def sol_exact(x):
    x_tem = torch.exp(x)
    sol = torch.sum(x_tem,dim=1).reshape([x.size()[0],1])
    return sol

In [5]:
def sol_exact_H1(x):
    x_tem = torch.exp(x)
    sol = torch.zeros(x.size()[0],3).cuda()
    sol[:,0] = torch.sum(x_tem,dim=1)
    sol[:,1:3] = torch.exp(x)
    return sol

In [6]:
def right(x):
    x_tem = torch.exp(x)
    f_tem = 2*torch.sum(x_tem,dim=1).reshape([x.size()[0],1])
    return f_tem 

In [7]:
dim_set = 2
Batch_size = 10000
def DRM1():
    x = torch.rand(Batch_size,dim_set).cuda()
    x.requires_grad = True
    u = model_u(x)
    du = model_p(x)
    v= torch.ones(u.shape).cuda()
    ux = torch.autograd.grad(u,x,grad_outputs=v,create_graph=True)[0]
    loss1 = torch.sum((ux-du)**2)/Batch_size
    return loss1

In [8]:
def DRM2():
    x = torch.rand(Batch_size,dim_set).cuda()
    x.requires_grad = True
    u = model_u(x)
    du = model_p(x)
    v= torch.ones(u.shape).cuda()
    uxx = torch.zeros(Batch_size,dim_set).cuda()
    for i in range(dim_set):
        ux_tem = du[:,i].reshape([x.size()[0],1])
        uxx_tem = torch.autograd.grad(ux_tem,x,grad_outputs=v,create_graph=True)[0]
        uxx[:,i] = uxx_tem[:,i]
    loss2 = torch.sum((torch.sum(uxx,dim=1).reshape([x.size()[0],1]) + u - right(x))**2)/Batch_size
    return loss2

In [13]:
def model_p(x):
    multi = x*(1-x)
    return model_p1(x)*multi + (exp(1)-1)*x+ 1

In [14]:
model_u = ResNet(dim_set,10,1)
model_p1 = ResNet(dim_set,10,dim_set)

device=torch.device("cuda:0" )
model_u.to(device)
model_p1.to(device)

ResNet(
  (fc1): Linear(in_features=2, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=10, bias=True)
  (outlayer): Linear(in_features=10, out_features=2, bias=True)
)

In [15]:
notes = torch.rand(10000,dim_set).cuda()
exact = sol_exact_H1(notes)
u_L2 = sqrt(torch.sum((exact[:,0])**2)/10000)
Du_L2 = sqrt(torch.sum(exact[:,1:3]**2)/10000)
def loss_error():
    notes = torch.rand(10000,dim_set).cuda()
    predict = torch.zeros(10000,2*dim_set+2).cuda()
    predict[:,0]= model_u(notes).reshape([1,notes.size()[0]])
    predict[:,1:dim_set+1]= model_p(notes)
    exact = sol_exact_H1(notes)
    value1 = sqrt(torch.sum((predict[:,0] - exact[:,0])**2)/10000)/u_L2
    value2 = sqrt(torch.sum((predict[:,1:3] - exact[:,1:3])**2)/10000)/Du_L2
    return value1,value2

In [16]:
traintime = 10000
error_save=np.zeros([2,traintime])
optimizer = optim.Adam([
                {'params': model_u.parameters()},
                {'params': model_p1.parameters()},
            ])
time_start=time.time()
for i in range(traintime):
    optimizer.zero_grad()
    losses1 = DRM1()
    losses1.backward()
    optimizer.step() 
    optimizer.zero_grad()
    losses2 = DRM2()
    losses2.backward()
    optimizer.step() 
    error1,error2 = loss_error()
    error_save[0,i]=float(error1)
    error_save[1,i]=float(error2)
    #scheduler.step()
    if i%500==1:
        print("i= ",i)
        print("error =",error1)
        print("loss1 =",losses1.detach())
        print("loss2 =",losses2.detach())
        np.save("error_save_LDG_NBC_2D_v2.npy", error_save)
np.save("error_save_LDG_NBC_2D_v2.npy", error_save)
time_end=time.time()
print('time cost',time_end-time_start,'s')

i=  1
error = 0.9039895167828308
loss1 = tensor(5.4447, device='cuda:0')
loss2 = tensor(11.5888, device='cuda:0')
i=  501
error = 0.0022050473275766055
loss1 = tensor(0.0027, device='cuda:0')
loss2 = tensor(0.0002, device='cuda:0')
i=  1001
error = 0.0008542436151226864
loss1 = tensor(0.0006, device='cuda:0')
loss2 = tensor(2.1963e-05, device='cuda:0')
i=  1501
error = 0.0005045889450742036
loss1 = tensor(0.0002, device='cuda:0')
loss2 = tensor(9.8467e-06, device='cuda:0')
i=  2001
error = 0.00860072854184548
loss1 = tensor(0.0051, device='cuda:0')
loss2 = tensor(6.7799e-05, device='cuda:0')
i=  2501
error = 0.0009446617444246497
loss1 = tensor(9.5950e-05, device='cuda:0')
loss2 = tensor(5.4411e-06, device='cuda:0')
i=  3001
error = 0.0016797558819508672
loss1 = tensor(0.0004, device='cuda:0')
loss2 = tensor(3.6416e-06, device='cuda:0')
i=  3501
error = 0.00018831800935196179
loss1 = tensor(3.0301e-05, device='cuda:0')
loss2 = tensor(7.7471e-07, device='cuda:0')
i=  4001
error = 0.0002

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
index = np.arange(0,50000,10)
plt.plot(np.log10(error_save[0,index]))